# Практическая работа 7 Журавлев А.Д. ББМО-02-23

# Фреймворк Adversarial Robustness Toolbox

Adversarial Robustness Toolbox (ART) v1.14 - это фреймворк, предназначенный для создания, анализа и обработки атак на модели машинного обучения. В этом ответе мы рассмотрим ключевые аспекты ART, выберем сценарий атаки и реализуем его на платформе Google Colab.

Генерация атак: ART предоставляет различные алгоритмы для генерации атак, такие как FGSM, BIM, PGD и другие.

# Fast Gradient Sign Method (FGSM)

Основная идея FGSM заключается в следующем:

*Определение функции потерь: *

Вычисляется функция потерь модели, которая измеряет, насколько хорошо модель предсказывает правильные метки для входных данных.

*Вычисление градиента: *

Вычисляется градиент функции потерь по отношению к входным данным. Этот градиент показывает направление, в котором нужно изменить входные данные, чтобы увеличить ошибку модели.

*Создание враждебного примера: *

Входные данные изменяются в направлении знака градиента на небольшую величину, определяемую параметром 𝜖 ϵ. Это изменение делает входные данные "враждебными" для модели, то есть такими, которые модель будет классифицировать неправильно.

Математическое представление: 𝑥 𝑎 𝑑 𝑣 = 𝑥 + 𝜖 ⋅ sign ( ∇ 𝑥 𝐽 ( 𝜃 , 𝑥 , 𝑦 ) ) x adv​=x+ϵ⋅sign(∇ x​J(θ,x,y))

# Проведение атаки с помощью сценария FGSM

In [2]:
!pip install adversarial-robustness-toolbox
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.6 MB/s eta 0:00:00


# Конвертация данных в формат PyTorch

In [3]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()
x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

# Создание модели

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Создание классификатора для ART

In [5]:
classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

# Создание атаки с помощью FGSM

In [6]:
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

In [7]:
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Точность: {:.2f}%".format(accuracy * 100))

Точность: 9.84%


# Обучение модели

In [8]:
from art.defences.trainer import AdversarialTrainer
adv_trainer = AdversarialTrainer(classifier, attack)

nb_epochs = 5
adv_trainer.fit(x_train, y_train, nb_epochs=nb_epochs)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/5 [00:00<?, ?it/s]

# Оценка модели после проведенного обучения на атакующих примерах

In [9]:
predictions_adv_trained = adv_trainer.predict(x_test_adv)
accuracy_adv_trained = np.sum(np.argmax(predictions_adv_trained, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Точность на атакующих данных после обучения: {:.2f}%".format(accuracy_adv_trained * 100))

Точность после обучения: 93.78%


# Оценка модели после проведенного обучения на чистых примерах

In [10]:
predictions_clean_trained = adv_trainer.predict(x_test)
accuracy_clean_trained = np.sum(np.argmax(predictions_clean_trained, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Точность на чистых данных после обучения: {:.2f}%".format(accuracy_clean_trained * 100))

Точность на чистых данных после обучения: 97.53%
